In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os

In [2]:
enron_df = pd.read_csv(os.path.join("enron_salary_numeric.csv"))
enron_df.head()

,name,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,...,loan_advances,from_messages,other,from_this_person_to_poi,poi,director_fees,deferred_income,long_term_incentive,email_address,from_poi_to_this_person
0,METTS MARK,365788.0,807.0,0.0,1061827.0,0.0,600000.0,585062.0,702.0,0.0,...,0.0,29.0,1740.0,1.0,False,0.0,0.0,0.0,mark.metts@enron.com,38.0
1,BAXTER JOHN C,267102.0,0.0,1295738.0,5634343.0,6680544.0,1200000.0,3942714.0,0.0,0.0,...,0.0,0.0,2660303.0,0.0,False,0.0,-1386055.0,1586055.0,0,0.0
2,ELLIOTT STEVEN,170941.0,0.0,0.0,211725.0,4890344.0,350000.0,1788391.0,0.0,0.0,...,0.0,0.0,12961.0,0.0,False,0.0,-400729.0,0.0,steven.elliott@enron.com,0.0
3,CORDES WILLIAM R,0.0,764.0,0.0,0.0,651850.0,0.0,386335.0,58.0,0.0,...,0.0,12.0,0.0,0.0,False,0.0,0.0,0.0,bill.cordes@enron.com,10.0
4,HANNON KEVIN P,243293.0,1045.0,0.0,288682.0,5538001.0,1500000.0,853064.0,1035.0,0.0,...,0.0,32.0,11350.0,21.0,True,0.0,-3117011.0,1617011.0,kevin.hannon@enron.com,32.0


In [3]:
# Drop Total Column
enron_df = enron_df[enron_df.name != 'TOTAL']

In [4]:
# Drop non numeric columns
drop_columns = ['name', 'email_address']
enron_df.drop(drop_columns, inplace=True, axis=1)
enron_df.head()

,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,expenses,loan_advances,from_messages,other,from_this_person_to_poi,poi,director_fees,deferred_income,long_term_incentive,from_poi_to_this_person
0,365788.0,807.0,0.0,1061827.0,0.0,600000.0,585062.0,702.0,0.0,585062.0,94299.0,0.0,29.0,1740.0,1.0,False,0.0,0.0,0.0,38.0
1,267102.0,0.0,1295738.0,5634343.0,6680544.0,1200000.0,3942714.0,0.0,0.0,10623258.0,11200.0,0.0,0.0,2660303.0,0.0,False,0.0,-1386055.0,1586055.0,0.0
2,170941.0,0.0,0.0,211725.0,4890344.0,350000.0,1788391.0,0.0,0.0,6678735.0,78552.0,0.0,0.0,12961.0,0.0,False,0.0,-400729.0,0.0,0.0
3,0.0,764.0,0.0,0.0,651850.0,0.0,386335.0,58.0,0.0,1038185.0,0.0,0.0,12.0,0.0,0.0,False,0.0,0.0,0.0,10.0
4,243293.0,1045.0,0.0,288682.0,5538001.0,1500000.0,853064.0,1035.0,0.0,6391065.0,34039.0,0.0,32.0,11350.0,21.0,True,0.0,-3117011.0,1617011.0,32.0


In [5]:
target = enron_df["poi"]
target_names = ["False", "True"]

In [6]:
data = enron_df.drop("poi", axis=1)
feature_names = data.columns
data.head()

,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,expenses,loan_advances,from_messages,other,from_this_person_to_poi,director_fees,deferred_income,long_term_incentive,from_poi_to_this_person
0,365788.0,807.0,0.0,1061827.0,0.0,600000.0,585062.0,702.0,0.0,585062.0,94299.0,0.0,29.0,1740.0,1.0,0.0,0.0,0.0,38.0
1,267102.0,0.0,1295738.0,5634343.0,6680544.0,1200000.0,3942714.0,0.0,0.0,10623258.0,11200.0,0.0,0.0,2660303.0,0.0,0.0,-1386055.0,1586055.0,0.0
2,170941.0,0.0,0.0,211725.0,4890344.0,350000.0,1788391.0,0.0,0.0,6678735.0,78552.0,0.0,0.0,12961.0,0.0,0.0,-400729.0,0.0,0.0
3,0.0,764.0,0.0,0.0,651850.0,0.0,386335.0,58.0,0.0,1038185.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,10.0
4,243293.0,1045.0,0.0,288682.0,5538001.0,1500000.0,853064.0,1035.0,0.0,6391065.0,34039.0,0.0,32.0,11350.0,21.0,0.0,-3117011.0,1617011.0,32.0


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

In [ ]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [8]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

NameError: name 'model' is not defined